<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Douglas Fabian Paris Merin & Noe Alejando Salinas Gandara.

**Fecha:** 03 de marzo del 2020.

**Expediente** : 714977 & 714748.
**Profesor:** Oscar David Jaramillo Zuluaga.

# Tarea 6: Clase10 MetodoAceptacionRechazo

## Resolver los siguientes problemas:

### Ejercicio 1: 
>Seguir un procedimiento similar al mostrado cuando se generó una distribución binomial, pero en esta caso genere un código que genere variables aletorias Poisson cuya función de distribución de probabilidad esta dada por:
>$$P(k,\lambda)=\frac{e^{-\lambda}(\lambda)^k}{k!}$$
    
> Demuestre matemáticamente que 
> $$P(k+1)=\frac{\lambda}{k+1}P(k)$$
> y a partir de esta relación genere variables aletorias que distribuyen poisson usando el método de la transformada inversa.


#### Código de solución 
Para facilitar la demostracion, pasare diviendo el $P_{(k)}$ de forma que nos queda 


\begin{equation*}
\begin{aligned}
\frac{P_{k+1}}{P_{k}} &= \frac{\lambda}{k+1} \\
\newline
P_{k+1} &= \frac{\lambda^{k+1} ~ e^{-\lambda}}{k+1!}\\
\newline
P_k &= \frac{\lambda^k e^{-\lambda}}{k!}\\
\newline
\frac{P_{k+1}}{P_k} &= \frac{\frac{e^{-\lambda}\lambda^{k+1}}{k+1!}}{\frac{e^{-\lambda}\lambda^k}{k!}}\\
\newline
&= \frac{k!~\underline{e^{-\lambda}}{\lambda^{k}+1}}{k+1!~\underline{e^{-\lambda}}\lambda^k}\\
\newline
&= \frac{k!~\underline{\lambda^{k+1}}}{k+1!~\underline{\lambda^k}}\\
\newline
&= \frac{k!~\lambda}{k+1!}\\
\newline
&= \frac{\underline{k!}~\lambda}{(k+1) \underline{k!}}\\
\newline
&= \frac{~\lambda}{(k+1)} = \frac{P_{k+1}}{P_{k}}
\end{aligned}
\end{equation*}

## Ejercicio 2
Partiendo que se desea generar variables aleatorias para la siguiente función de densidad
$$f(x)=30(x^2-2x^3+x^4)$$
Responda los siguientes literales:
1. Usar como función que mayora a $f(x)$ a $t(x)=a \sin(\pi x)$ donde a es el máximo de la función $f(x)$ y graficarlas en una misma gráfica, para validar que en realidad si cumple la condición $t(x)\geq f(x)$.
2. Encontrar la función de densidad $g(x)$ según lo visto en clase. Reportar todos los cálculos realizados para encontrar dicha función usando Markdown (Latex).
3. Usar la función encontrada en el punto 2 y utilizar el método de la transformada inversa visto en la clase 9, para generar variables aleatorias que sigan la distribución $g(x)$. **Nota:** Recuerde que el método de la transformada inversa funciona con la distribución de probabilidad acumulada y no con su densidad. Nuevamente similar al punto anterior reportar todos los cálculos usando Markdown (Latex). 
4. Según el punto 3, generar 10000 puntos aleatorios que sigan la distribución $g(x)$ y comparar con su histograma para validar que los puntos generados siguen la distribución deseada.